In [7]:
cd ..

/home/user/Documents/workspace/projects/accident-prediction-montreal


In [10]:
export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"

SyntaxError: invalid syntax (<ipython-input-10-23657fe0ba52>, line 1)

In [8]:
import sys
sys.path.insert(0, '/home/user/Documents/workspace/spark/python')

In [9]:
from utils import init_spark
spark = init_spark()

Exception: Java gateway process exited before sending its port number

In [ ]:
from pyspark.sql.functions import *
df = spark.createDataFrame(zip(range(50000),))
df = df.select(randn().alias('x'), randn().alias('y'))
df = df.withColumn('label', when(col('x') < col('y'), lit(1.0)).otherwise(lit(0.0)))
df = df.filter(col('label') == 0).union(df.filter(col('label') == 1.0).sample(1/5))
df.toPandas().plot.scatter(x='x', y="y", c="label", colormap="viridis")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from class_weighter import ClassWeighter
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


(train_set, test_set) = df.randomSplit([0.7, 0.3])

In [ ]:
va = VectorAssembler(inputCols=['x', 'y'], outputCol="features")
rf = RandomForestClassifier(labelCol="label",
                            featuresCol="features",
                            cacheNodeIds=True,
                            maxDepth=30,
                            weightCol='weight')
cw_weighted = ClassWeighter().setClassWeight([1/5, 1.0])
cw = ClassWeighter().setClassWeight([1.0, 1.0])
pipeline_weighted = Pipeline().setStages([va, cw_weighted, rf])
pipeline = Pipeline().setStages([va, cw, rf])
model = pipeline.fit(train_set)
model_weighted = pipeline_weighted.fit(train_set)
pred = model.transform(test_set)
pred_weighted = model_weighted.transform(test_set)

In [ ]:
pred.toPandas().plot.scatter(x='x', y="y", c="prediction", colormap="viridis")
pred_weighted.toPandas().plot.scatter(x='x', y="y", c="prediction", colormap="viridis")

In [ ]:
fn = pred.filter((col('prediction') == 0.0) & (col('label') == 1.0)).count()
fn_weighted = pred_weighted.filter((col('prediction') == 0.0) & (col('label') == 1.0)).count()
print(fn, fn_weighted)

In [ ]:
fp = pred.filter((col('prediction') == 1.0) & (col('label') == 0.0)).count()
fp_weighted = pred_weighted.filter((col('prediction') == 1.0) & (col('label') == 0.0)).count()
print(fp, fp_weighted)

In [ ]:
from evaluate import evaluate_binary_classifier
evaluate_binary_classifier(pred)
evaluate_binary_classifier(pred_weighted)